<a href="https://colab.research.google.com/github/inagib21/BuildingPermitAnalysis/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.express as px

# Data Understanding:
**Cb No** Community Board

In [2]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!gdown --id 16dH6ZQFAmIOAK4CWXfhn60fUVoGSbVkz

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16dH6ZQFAmIOAK4CWXfhn60fUVoGSbVkz
To: /content/ApprovedPermits.csv
100% 213M/213M [00:04<00:00, 52.1MB/s]


In [4]:
# Read CSV file into a DataFrame
df = pd.read_csv('ApprovedPermits.csv')

<ipython-input-4-f5cea7a8267f>:2: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ApprovedPermits.csv')


In [5]:
#inspecting Dimensions
df.shape

(445019, 35)

In [6]:
#checking column info aswell as missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445019 entries, 0 to 445018
Data columns (total 35 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Job Filing Number                     445019 non-null  object 
 1   Filing Reason                         445019 non-null  object 
 2   House No                              445019 non-null  object 
 3   Street Name                           445019 non-null  object 
 4   Borough                               445019 non-null  object 
 5   LOT                                   444877 non-null  float64
 6   Bin                                   445019 non-null  int64  
 7   Block                                 445019 non-null  int64  
 8   C B NO                                444759 non-null  float64
 9   Apt/Condo No(s)                       1710 non-null    object 
 10  Work on Floor                         443100 non-null  object 
 11  

it seems that some of the columns in the DataFrame have no non-null values, which means that they contain no useful information for analysis. Here are the columns that have no non-null values:

Owner Street Address
Owner City
Owner State
Owner Zip Code
Filing Representative Middle Initial

In [7]:
#Dropping columns with no values
df = df.drop(['Owner Street Address', 'Owner City', 'Owner State', 'Owner Zip Code', 'Filing Representative Middle Initial'], axis=1)


In [8]:
#Dropping columns with very small amount of values
df = df.drop(['Apt/Condo No(s)','Applicant Middle Name'],axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445019 entries, 0 to 445018
Data columns (total 28 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Job Filing Number                    445019 non-null  object 
 1   Filing Reason                        445019 non-null  object 
 2   House No                             445019 non-null  object 
 3   Street Name                          445019 non-null  object 
 4   Borough                              445019 non-null  object 
 5   LOT                                  444877 non-null  float64
 6   Bin                                  445019 non-null  int64  
 7   Block                                445019 non-null  int64  
 8   C B NO                               444759 non-null  float64
 9   Work on Floor                        443100 non-null  object 
 10  Work Type                            445019 non-null  object 
 11  Permittee's L

Almost half of the dataset does not have a filing representative. instead of dropping the rows that do not have any values we will fill the First name, Last Name and Business name with 'Unknown'.


In [10]:
df['Filing Representative Business Name'] = df['Filing Representative Business Name'].fillna('Unknown')


In [11]:
df['Filing Representative First Name'] = df['Filing Representative First Name'].fillna('Unknown')

In [12]:
df['Filing Representative Last Name'] = df['Filing Representative Last Name'].fillna('Unknown')

In [13]:
# Check for NaN values
print(df.isnull().sum())

Job Filing Number                          0
Filing Reason                              0
House No                                   0
Street Name                                0
Borough                                    0
LOT                                      142
Bin                                        0
Block                                      0
C B NO                                   260
Work on Floor                           1919
Work Type                                  0
Permittee's License Type                   0
Applicant License #                        0
Applicant First Name                       1
Applicant Last Name                        2
Applicant Business Name                  333
Applicant Business Address             26622
Filing Representative First Name           0
Filing Representative Last Name            0
Filing Representative Business Name        0
Work Permit                                0
Approved Date                             30
Issued Dat

In [14]:
df['Applicant Business Address'] = df['Applicant Business Address'].fillna('Unknown')
df['Owner Business Name'] = df['Owner Business Name'].fillna('Unknown')


In [15]:
#Now we can drop any rows with missiing values
df = df.dropna()

In [16]:
df.shape

(442248, 28)

In [17]:
# Now we make sure columns are correct df type
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442248 entries, 0 to 445018
Data columns (total 28 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Job Filing Number                    442248 non-null  object 
 1   Filing Reason                        442248 non-null  object 
 2   House No                             442248 non-null  object 
 3   Street Name                          442248 non-null  object 
 4   Borough                              442248 non-null  object 
 5   LOT                                  442248 non-null  float64
 6   Bin                                  442248 non-null  int64  
 7   Block                                442248 non-null  int64  
 8   C B NO                               442248 non-null  float64
 9   Work on Floor                        442248 non-null  object 
 10  Work Type                            442248 non-null  object 
 11  Permittee's L

In [18]:
# Convert date variables to datetime format
df['Approved Date'] = pd.to_datetime(df['Approved Date'])
df['Issued Date'] = pd.to_datetime(df['Issued Date'])
df['Expired Date'] = pd.to_datetime(df['Expired Date'])


In [19]:
# Extract features from date variables
df['Approved Year'] = df['Approved Date'].dt.year
df['Approved Month'] = df['Approved Date'].dt.month
df['Issued Year'] = df['Issued Date'].dt.year
df['Issued Month'] = df['Issued Date'].dt.month
df['Expired Year'] = df['Expired Date'].dt.year
df['Expired Month'] = df['Expired Date'].dt.month


In [20]:

# create correlation matrix
corr = df.corr()

fig = px.imshow(df.corr(), color_continuous_scale='RdBu', title='Correlation Heatmap')
fig.update_layout(template='plotly_dark', paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', )
fig.show()


<ipython-input-20-480051ca1f52>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()
<ipython-input-20-480051ca1f52>:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  fig = px.imshow(df.corr(), color_continuous_scale='RdBu', title='Correlation Heatmap')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442248 entries, 0 to 445018
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Filing Reason                        442248 non-null  object        
 1   House No                             442248 non-null  float64       
 2   Street Name                          442248 non-null  object        
 3   Borough                              442248 non-null  object        
 4   LOT                                  442248 non-null  float64       
 5   Block                                442248 non-null  int64         
 6   C B NO                               442248 non-null  float64       
 7   Work on Floor                        442248 non-null  object        
 8   Work Type                            442248 non-null  object        
 9   Permittee's License Type             442248 non-null  object        
 

In [21]:
print(df['Bin'].nunique())

83331


In [22]:
print(df['C B NO'].nunique())

71


In [23]:
# Because bin and Community Board are highly correlated lets drop bin
df = df.drop('Bin', axis=1)

In [24]:
# Now Lets convert our object columns that should be numbers
df['C B NO'] = pd.to_numeric(df['C B NO'], errors='coerce')
df['House No'] = pd.to_numeric(df['House No'], errors='coerce')


In [25]:
# Check for NaN values
print(df.isnull().sum())

Job Filing Number                          0
Filing Reason                              0
House No                               77950
Street Name                                0
Borough                                    0
LOT                                        0
Block                                      0
C B NO                                     0
Work on Floor                              0
Work Type                                  0
Permittee's License Type                   0
Applicant License #                        0
Applicant First Name                       0
Applicant Last Name                        0
Applicant Business Name                    0
Applicant Business Address                 0
Filing Representative First Name           0
Filing Representative Last Name            0
Filing Representative Business Name        0
Work Permit                                0
Approved Date                              0
Issued Date                                0
Expired Da

In [26]:
# We dont need this column
df = df.drop('Job Filing Number', axis=1)

In [27]:
df['House No'] =df['House No'].fillna(0)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442248 entries, 0 to 445018
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Filing Reason                        442248 non-null  object        
 1   House No                             442248 non-null  float64       
 2   Street Name                          442248 non-null  object        
 3   Borough                              442248 non-null  object        
 4   LOT                                  442248 non-null  float64       
 5   Block                                442248 non-null  int64         
 6   C B NO                               442248 non-null  float64       
 7   Work on Floor                        442248 non-null  object        
 8   Work Type                            442248 non-null  object        
 9   Permittee's License Type             442248 non-null  object        
 

In [34]:
unique_work_types = df['Work Type'].unique()
print(unique_work_types)


['Sidewalk Shed' 'Construction Fence' 'Foundation' 'General Construction'
 'Mechanical Systems' 'Earth Work' 'Protection and Mechanical Methods'
 'Supported Scaffold' 'Sign' 'Plumbing' 'Sprinklers' 'Boiler Equipment'
 'Antenna' 'Structural' 'Support of Excavation' 'Curb Cut' 'Standpipe']


In [35]:
relevant_work_types = [
    "Foundation",
    "General Construction",
    "Mechanical Systems",
    "Earth Work",
    "Structural",
    "Support of Excavation",
]

filtered_df = df[df["Work Type"].isin(relevant_work_types)]

In [36]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146491 entries, 2 to 445017
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Filing Reason                        146491 non-null  object        
 1   House No                             146491 non-null  float64       
 2   Street Name                          146491 non-null  object        
 3   Borough                              146491 non-null  object        
 4   LOT                                  146491 non-null  float64       
 5   Block                                146491 non-null  int64         
 6   C B NO                               146491 non-null  float64       
 7   Work on Floor                        146491 non-null  object        
 8   Work Type                            146491 non-null  object        
 9   Permittee's License Type             146491 non-null  object        
 

In [68]:
# Aggregate the data
grouped_data = filtered_df['Borough'].value_counts().reset_index(name='Count')
grouped_data.columns = ['Borough', 'Count']

# Create the bar chart using Plotly Express
fig = px.bar(grouped_data, x='Borough', y='Count', title='Permits Issued by Borough')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()


In [60]:
# Aggregate the data
grouped_data = filtered_df.groupby(['Borough', 'Work Type']).size().reset_index(name='Count')

# Create the bar chart using Plotly Express
fig = px.bar(grouped_data, x='Work Type', y='Count', color='Borough', title='Permits Issued by Work Type and Borough')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()


In [61]:
# Aggregate the data
grouped_data = filtered_df.groupby('Work Type')['Estimated Job Costs'].mean().reset_index(name='Average Estimated Job Costs')

# Create the bar chart using Plotly Express
fig = px.bar(grouped_data, x='Work Type', y='Average Estimated Job Costs', title='Average Estimated Job Costs by Work Type')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()



In [63]:
# Aggregate the data
grouped_data = filtered_df.groupby(['Approved Year', 'Approved Month']).size().reset_index(name='Count')

# Create a new column for year-month combination
grouped_data['Year-Month'] = grouped_data['Approved Year'].astype(str) + '-' + grouped_data['Approved Month'].astype(str)

# Create the line chart using Plotly Express
fig = px.line(grouped_data, x='Year-Month', y='Count', title='Total Permits Issued by Month and Year')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()


In [64]:
# Aggregate the data
grouped_data = filtered_df.groupby(['Approved Year', 'Approved Month', 'Work Type']).size().reset_index(name='Count')

# Create a new column for year-month combination
grouped_data['Year-Month'] = grouped_data['Approved Year'].astype(str) + '-' + grouped_data['Approved Month'].astype(str)

# Create the line chart using Plotly Express
fig = px.line(grouped_data, x='Year-Month', y='Count', color='Work Type', title='Permits Issued by Work Type and Month-Year')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()


In [66]:
# Aggregate the data
grouped_data = filtered_df.groupby('Work Type')['Estimated Job Costs'].sum().reset_index(name='Total Estimated Job Costs')

# Create the bar chart using Plotly Express
fig = px.bar(grouped_data, x='Work Type', y='Total Estimated Job Costs', title='Total Estimated Job Costs by Work Type')

# Customize the chart layout
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='black'),
    xaxis=dict(showgrid=True, gridcolor='black'),
    yaxis=dict(showgrid=True, gridcolor='black')
)

# Show the chart
fig.show()


In [69]:
boroughs = filtered_df['Borough'].unique()

for borough in boroughs:
    # Filter the data for the current borough
    borough_data = filtered_df[filtered_df['Borough'] == borough]
    
    # Aggregate the data
    grouped_data = borough_data['Work Type'].value_counts().reset_index(name='Count')
    grouped_data.columns = ['Work Type', 'Count']

    # Create the bar chart using Plotly Express
    fig = px.bar(grouped_data, x='Work Type', y='Count', title=f'Permits Issued by Work Type in {borough}')

    # Customize the chart layout
    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        font=dict(color='black'),
        xaxis=dict(showgrid=True, gridcolor='black'),
        yaxis=dict(showgrid=True, gridcolor='black')
    )

    # Show the chart
    fig.show()


predicting the number of permits issued for a particular work type in the upcoming months.

In [78]:
# Create a new column 'Approved Month-Year' by combining Approved Month and Approved Year
filtered_df['Approved Month-Year'] = pd.to_datetime(filtered_df['Approved Year'].astype(str) + '-' + filtered_df['Approved Month'].astype(str))

# Group the data by Work Type and Approved Month-Year, and count the number of permits issued
monthly_data = filtered_df.groupby(['Work Type', 'Approved Month-Year']).size().reset_index(name='Count')



<ipython-input-78-3d8486370e3c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
# Pivot the data to create a time series for each work type
time_series_data = monthly_data.pivot_table(index='Approved Month-Year', columns='Work Type', values='Count', fill_value=0)


In [101]:
# Calculate the simple moving average with a window size of 3 months
sma = time_series_data.rolling(window=3).mean().reset_index()



In [102]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_size = int(len(time_series_data) * 0.8)
train_data, test_data = time_series_data.iloc[:train_size], time_series_data.iloc[train_size:]


In [87]:
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [97]:

for work_type in relevant_work_types:
    print(work_type, sma_relevant[work_type].dtype)


Foundation float64
General Construction float64
Mechanical Systems float64
Earth Work float64
Structural float64
Support of Excavation float64


In [105]:
# Select the relevant work types and their rolling averages from the sma DataFrame
sma_relevant = sma[relevant_work_types]

# Get the last available date in the sma DataFrame
last_date = sma_relevant.index[-1]

# Create an empty DataFrame to store the predictions
predictions = pd.DataFrame(index=test_data.index, columns=relevant_work_types)

# Loop through each work type and use the last available value as the prediction
for work_type in relevant_work_types:
    last_value = sma_relevant.loc[last_date, work_type]
    predictions[work_type] = np.full_like(predictions[work_type], last_value)



In [106]:
# Calculate the evaluation metrics (MAE and MSE) for each work type
mae_scores = {}
mse_scores = {}

for work_type in relevant_work_types:
    mae_scores[work_type] = mean_absolute_error(test_data[work_type], predictions[work_type])
    mse_scores[work_type] = mean_squared_error(test_data[work_type], predictions[work_type])

print("MAE Scores:")
print(mae_scores)

print("MSE Scores:")
print(mse_scores)

MAE Scores:
{'Foundation': 108.9, 'General Construction': 1099.1, 'Mechanical Systems': 410.8666666666666, 'Earth Work': 62.6, 'Structural': 233.7, 'Support of Excavation': 29.2}
MSE Scores:
{'Foundation': 14899.5, 'General Construction': 1501686.5, 'Mechanical Systems': 207162.31111111108, 'Earth Work': 5310.711111111112, 'Structural': 68734.14444444444, 'Support of Excavation': 1076.1111111111109}


the MAE (Mean Absolute Error) measures the average absolute difference between the predicted values and the true values for each work type. So, a smaller MAE value indicates a better fit of the model.

The MSE (Mean Squared Error) measures the average squared difference between the predicted values and the true values for each work type. So, a smaller MSE value indicates a better fit of the model, and the values are on a larger scale than MAE.

Looking at the results, we can see that the Earth Work and Support of Excavation work types have the lowest MAE scores, indicating that the model was able to predict these types of work more accurately. On the other hand, the General Construction work type has the highest MAE score, indicating that the model was less accurate in predicting this type of work.

For MSE scores, we can see that the Earth Work and Support of Excavation work types have the lowest MSE scores, indicating that the model was able to predict these types of work more accurately with smaller error. However, General Construction work type has the highest MSE score, indicating that the model was less accurate in predicting this type of work with larger error.